# HW1: Frame-Level Speech Recognition

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 


In [4]:
!nvidia-smi

Sat Sep 14 03:29:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install --upgrade pip
!pip install torchsummaryX==1.1.0 wandb --quiet
!pip install slugify

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"jiayihuang2001","key":"d27e2930f4739944e1c7f6f2e539e1bc"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -c 11785-hw1p2-f24
!unzip -qo ./11785-hw1p2-f24.zip



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
  Preparing metadata (setup.py) ... done
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1883 sha256=0b52f5359c8de651e5f3e3b593c677eef1dfefc8cfa6e173050f7f5e46b1f693
  Stored in directory: /root/.cache/pip/wheels/17/4e/33/690b6dceba620cd40ba592d448adf2dd98cb5310355d7a448a
Successfully built slugify
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73249 sha256=7e58dfd64bc76795fe13b3014a13d81b3f03eb5306185b9fea2c530091c91708
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.17
    Uninstalling kaggle-1.6.17:
      Successfull

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [6]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [7]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

# Dataset

In [8]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100",fraction = 1): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        content_dir =os.path.join(root,"11785-f24-hw1p2")
        partition_dir = os.path.join(content_dir,partition)
        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = os.path.join(partition_dir,"mfcc")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(partition_dir,"transcript")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_list = sorted(os.listdir(self.mfcc_dir))
        mfcc_names = mfcc_list[0:int(fraction*len(mfcc_list))]
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcipt_list = sorted(os.listdir(self.transcript_dir))
        transcript_names = transcipt_list[0:int(fraction*len(transcipt_list))]

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []
        idx = 0
        file_idx_dict ={}
        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_n = mfcc - np.mean(mfcc,axis=0,keepdims=True)
        #   Load the corresponding transcript
            self.mfccs.append(mfcc_n)

        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir,transcript_names[i]))[1:-1] # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.transcripts.append(transcript)

        self.mfccs = np.concatenate(self.mfccs)
        self.mfccs = np.pad(self.mfccs,((context,context),(0,0)),mode ="constant")
        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate
        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.transcripts)  # Ensure length is based on transcripts

        phoneme_dict ={}
        for index,ph in enumerate(self.phonemes):
            phoneme_dict[ph]=index
        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.transcripts = np.array([phoneme_dict[ph] for ph in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        start_ind = ind
        end_ind = ind + 2*self.context +1
        # Retrieve the frames with context, handling potential edge cases
        frames = self.mfccs[start_ind:end_ind]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames     = torch.FloatTensor(frames) # Convert to tensors
        phonemes   = torch.tensor(self.transcripts[ind])

        return frames, phonemes



class AudioTestDataset(torch.utils.data.Dataset):
    def __init__(self, root, context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context = context
        content_dir =os.path.join(root,"11785-f24-hw1p2")
        partition_dir = os.path.join(content_dir,partition)
        self.mfcc_dir = os.path.join(partition_dir,"mfcc")
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        self.mfccs = []

        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir,mfcc_names[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc_n = mfcc - np.mean(mfcc,axis=0,keepdims=True)
            self.mfccs.append(mfcc_n)
        #   Load the corresponding transcript
        self.mfccs = np.concatenate(self.mfccs)
        self.length = len(self.mfccs)
        self.mfccs = np.pad(self.mfccs,((context,context),(0,0)),mode ="constant")

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        start_ind = ind
        end_ind = ind + 2*self.context +1
        # Retrieve the frames with context, handling potential edge cases
        frames = self.mfccs[start_ind:end_ind]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data
        frames     = torch.FloatTensor(frames) # Convert to tensors
        return frames


This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

# Create Datasets

In [9]:
config = {
    "model"         : "8-MLP",
    'epochs'        : 100,
    'batch_size'    : 1024,
    'context'       : 20,
    'init_lr'       : 0.00372,
    "dropout_rate"  : 0.5,
    'architecture'  : '8_layer_diamond'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [10]:

root = "/content/hw1"
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(root,context=config["context"])
# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data =  AudioDataset(root,context=config["context"],partition="dev-clean")
# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root,context=config["context"])

FileNotFoundError: [Errno 2] No such file or directory: '/content/hw1/11785-f24-hw1p2/train-clean-100/mfcc'

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

In [ ]:
# Testing code to check if your data loaders are working

# so only one batch here
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size,output_size,dropout):

        super(Network, self).__init__()
        size = 1024

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, size*2),
            torch.nn.BatchNorm1d(size*2),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size*2,size*3),
            torch.nn.BatchNorm1d(size*3),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size*3, size*2),
            torch.nn.BatchNorm1d(size*2),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size*2, size),
            torch.nn.BatchNorm1d(size),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size,size//2),
            torch.nn.BatchNorm1d(size//2),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size//2,size//4),
            torch.nn.BatchNorm1d(size//4),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size//4,size//8),
            torch.nn.BatchNorm1d(size//8),
            torch.nn.SiLU(),
            torch.nn.Dropout(p=dropout),
            torch.nn.Linear(size//8, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes),float(config["dropout_rate"])).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate,
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max= config['epochs'])
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="bc022b99e5a39b97fc6ae8c641ab328e9f52d2e6") #API Key is in your wandb account, under settings (wandb.ai/settings)

In [ ]:
# # Create your wandb run
# run = wandb.init(
#     name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
#     reinit  = True, ### Allows reinitalizing runs when you re-run this cell
#     #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
#     #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
#     project = "hw1p2", ### Project should be created in your wandb account
#     config  = config ### Wandb Config for your run
# )

In [ ]:
run = wandb.init(
    project="hw1p2",
    job_type="model-training",
    name=config['model'],
    config=config
    )
## Retreiving the model

# Getting the latest version of the artifact
artifact = run.use_artifact('{}:latest'.format(config['model']))
# Downloading the artifact
artifact_dir = artifact.download()
# Loading the model
model_dict = torch.load(os.path.join(artifact_dir, 'model_checkpoint.pth'))

# Loading weights
model.load_state_dict(model_dict['model_state_dict'])
# Loading optimizer state
optimizer.load_state_dict(model_dict['optimizer_state_dict'])
scheduler.load_state_dict(model_dict['scheduler_state_dict'])

In [ ]:
# latest_checkpoint = 'model_checkpoint.pth'
# if os.path.isfile(latest_checkpoint):
#     load_history = 'True'
# else:
#     load_history = 'False'
# if load_history == 'True':
#     checkpoint = torch.load(latest_checkpoint)
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

# summary(model, frames.to(device))

In [ ]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
# load_history =False
# if load_history == 'True':
#     checkpoint = torch.load(latest_checkpoint)
#     best_acc = checkpoint['acc']
# else:
#     best_acc = 0  ### Monitor best accuracy in your run
best_acc = 0
for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)
    scheduler.step()
    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    # Updating the model version
     ### Save checkpoint if accuracy is better than your current best
    if val_acc >= best_acc:
        ### Save checkpoint with information you want
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'train_loss': train_loss,
                    'val_acc': val_acc,
                    'train_acc': train_acc,
                    'valid_loss': val_loss
                    },
                    './model_checkpoint.pth')

        best_acc = val_acc

        ### Save checkpoint in wandb
        wandb.save('checkpoint.pth')
        model_artifact = wandb.Artifact(config['model'], type='model')
        # Adding model file to Artifact
        model_artifact.add_file("model_checkpoint.pth")
        # Saving Artifact to WandB
        run.log_artifact(model_artifact)


    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits,dim=1)
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.append(predicted_phonemes)
            # TODO
        print(len(torch.concatenate(test_predictions)))
    return torch.concatenate(test_predictions)

In [ ]:
predictions = test(model, test_loader)

In [ ]:
print(predictions.shape)

In [ ]:
### Create CSV file with predictions

with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, PHONEMES[predictions[i]]))

In [ ]:
### Finish your wandb run
run.finish()

In [ ]:
# ### Submit to kaggle competition using kaggle API (Uncomment below to use)
# !kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

# ### However, its always safer to download the csv file and then upload to kaggle